# GPT2SP - Model Inspection Notebook

#### Hello!  Welcome to the model inspection notebook, we provides the scripts for model loading, model inference on testing data, and tensorboard of model training process for all of the models mentioned in our experiments in the paper. 

##### Attention!!!
##### Before interacting with this notebook, you may want to install a few dependencies [HERE](#dependencies).
##### Also, make sure to run the [Static Methods](#static-method) cell, then you are good to go
#### The models are categorized by the experiment scenario, please follow the link as follows to reach the specific section

### 1. [Within Project Models](#within_project)
### 2. [Cross Project Models](#cross_project)

<a id='dependencies'></a>
## Dependencies Installation
#### run the cell below to install the dependencies

In [ ]:
!pip install transformers
!pip install torch
!pip install pandas
!pip install numpy
!pip install webbrowser
!pip install tokenizers

<a id='static-method'></a>
## Static Methods

In [45]:
from transformers import GPT2Config
import torch
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import GPT2Tokenizer
import numpy as np
import time
from GPT2SP import GPT2ForSequenceClassification as GPT2SP
import webbrowser
from tokenizers import Tokenizer
# local import
from custom_transformers_interpret.explainers import SequenceClassificationExplainer

# TensorBoard Mapping 
TENSORBOARD_MAPPING = {"#0": "PXtBXB2lTaWHOt7Ba8Lo8A", "#00": "YrgwheHpTeWBIxQjabrtjg", "#000": "IYJBq5aZT8emJn5NDF03nw", 
                      "#2": "0lzJAIahQOemygzRYDGcQQ", "#22": "JIV9lLB2RXSdyiZxkeRjug", "#222": "xQYRbZQhQfa4u2Mec7Zg8w",
                      "#6": "LBGQqy7LTlqjRlHeqjn0Ng", "#66": "nExRanq1Q9CwRxoesPjQWw", "#666": "31KJQ9FqTzip3JaHrrvxhA",
                      "#7": "c3zgX3j9T6S6qZYV4EdpEw", "#77": "gf5ErnJKSrCKY7p9z8il6w", "#777": "Vf5qkk4URnC97jd5vYrRzQ"}
# Tokenizer Mapping
TOKENIZER_MAPPING = {"#0": "gpt2_bpe", "#00": "gpt2_bpe", "#000": "gpt2_bpe", 
                      "#2": "sp_word_level", "#22": "sp_word_level", "#222": "sp_word_level",
                      "#6": "wordpiece_sp", "#66": "wordpiece_sp", "#666": "wordpiece_sp",
                      "#7": "sentencepiece_sp", "#77": "sentencepiece_sp", "#777": "sentencepiece_sp"}

# pad token ID mapping
PAD_TOKEN_ID_MAPPING = {"gpt2_bpe": 50256, "sp_word_level": 3, "wordpiece_sp": 0, "sentencepiece_sp": 0}
# static global vars
global DYNAMIC_BATCH, DEVICE
DYNAMIC_BATCH = True
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
SEQUENCE_LEN = 20
# dynamic global vars
global PAD_TOKEN_ID, BATCH_SIZE_RATIO, BATCH_SIZE, TOKENIZER, WITHIN_PROJECT, TEXT, KEY, TOK
PAD_TOKEN_ID = None
BATCH_SIZE_RATIO = None
BATCH_SIZE = None
TOKENIZER = None
WITHIN_PROJECT = None
TEXT = None
KEY = None
TOK = None

def tokenization(text_list):
    global TOKENIZER, SEQUENCE_LEN, MODEL, TOK
    # tokenization
    if TOKENIZER == 'sp_word_level':
        print('using word-level tokenizer!')
        tokenizer = Tokenizer.from_file('all_tokenizers/word_level/wordlevel.json')
        encoded_sentences = {'input_ids':[]}
        for sentence in text_list:
            encoded = tokenizer.encode(sentence)
            encoded = encoded.ids
            if len(encoded) > SEQUENCE_LEN:
                encoded = encoded[:SEQUENCE_LEN]
            elif len(encoded) < SEQUENCE_LEN:
                padding = SEQUENCE_LEN - len(encoded)
                for _ in range(padding):
                    encoded.append(3)
            encoded_sentences['input_ids'].append(encoded)
        return encoded_sentences
    elif TOKENIZER == 'gpt2_bpe':
        print('using GPT2 BPE tokenizer')
        tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        tokenizer.pad_token = '[PAD]'
    elif TOKENIZER == 'wordpiece_sp':
        print('using wordpiece tokenizer!')
        tokenizer = BertTokenizer('all_tokenizers/word_piece/vocab.txt')
    elif TOKENIZER == 'sentencepiece_sp':
        print('using sentencepiece tokenizer!')
        tokenizer = XLNetTokenizer('all_tokenizers/sentence_piece/spm_tokenizer.model', padding_side='right')
    TOK = tokenizer
    return tokenizer.batch_encode_plus(text_list, truncation=True, max_length=SEQUENCE_LEN, padding='max_length')


def prepare_dataframe(file_name):
    data = pd.read_csv(file_name)
    # some rows have no description, fill blank to avoid Null
    data = data.fillna(' ')
    d = {'text': data['title'], 'label': data['storypoint'], 'issuekey': data['issuekey']}
    return pd.DataFrame(data=d)


def prepare_dataloader(seq, y, sampler_type):
    global BATCH_SIZE
    tensor_dataset = TensorDataset(seq, y)
    if sampler_type == 'random':
        sampler = RandomSampler(tensor_dataset)
    elif sampler_type == 'sequential':
        sampler = SequentialSampler(tensor_dataset)
    dataloader = DataLoader(tensor_dataset, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader


def load_trained_model(model_id, project_name):
    global WITHIN_PROJECT 
    if WITHIN_PROJECT:
        path = "MickyMike/" + model_id[1:] + "-GPT2SP-" + project_name
    else:
        path = "MickyMike/" + model_id[1:] + "-GPT2SP-" + project_name.split("_")[0] + "-" + project_name.split("_")[1]
    return GPT2SP.from_pretrained(path)


def prepare_test_dataloader(file_name):
    global WITHIN_PROJECT, BATCH_SIZE, BATCH_SIZE_RATIO, TEXT, KEY
    if WITHIN_PROJECT:
        # calculate the batch size
        df = prepare_dataframe(file_name)
        BATCH_SIZE = int(int(len(df['text'][:int(len(df)*0.6)])) * BATCH_SIZE_RATIO)
        print("Batch Size: ", BATCH_SIZE)
        # prepare testing data
        test_text = df['text'][int(len(df)*0.8):]
        TEXT = test_text
        KEY = df['issuekey'][int(len(df)*0.8):]
        test_labels = df['label'][int(len(df)*0.8):]
        tokens_test = tokenization(test_text.tolist())
        test_seq = torch.tensor(tokens_test['input_ids'])
        test_y = torch.tensor(test_labels.tolist()).type(torch.LongTensor)
        test_dataloader = prepare_dataloader(test_seq, test_y, sampler_type='sequential')
    else:
        # calculate the batch size based on training data
        df = prepare_dataframe('sp_dataset/marked_training_data/' + file_name[0] + '.csv')
        BATCH_SIZE = int(int(len(df['text'][:int(len(df)*0.6)])) * BATCH_SIZE_RATIO)
        # prepare testing data
        df = prepare_dataframe('sp_dataset/marked_training_data/' + file_name[1] + '.csv')
        test_text = df['text']
        TEXT = test_text
        KEY = df['issuekey']
        test_labels = df['label']
        tokens_test = tokenization(test_text.tolist())
        test_seq = torch.tensor(tokens_test['input_ids'])
        test_y = torch.tensor(test_labels.tolist()).type(torch.LongTensor)
        test_dataloader = prepare_dataloader(test_seq, test_y, sampler_type='sequential')         
    return test_dataloader
  
    
def do_inference(trained_model, test_dataloader):
    global TEXT, KEY
    global XAI
    cls_explainer = SequenceClassificationExplainer(trained_model, TOK)
    predictions = []
    true_labels = []
    for batch in test_dataloader:
        batch = tuple(t.to(DEVICE) for t in batch)
        b_input_ids, b_labels = batch
        with torch.no_grad():
            logits = trained_model(b_input_ids)
        logits = logits['logits'].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.append(logits)
        true_labels.append(label_ids)    
    # calculate errors
    total_distance = 0
    index = 0
    for i in range(len(predictions)):
        for j in range(len(predictions[i])):
            # for each correctly predicted issue
            if round(predictions[i][j][0], 0) == true_labels[i][j] and XAI is True:
                # store every attr scores
                word_score_map = {}
                attr_scores = []
                top_words = []
                print("prediction: ", predictions[i][j])
                print("true label: ", true_labels[i][j])
                global TEXT, KEY
                print("Issue Key: ", KEY.iloc[index])
                attrs = cls_explainer(TEXT.iloc[index], ground_truth=true_labels[i][j])                
                print("Attribute Values: ", attrs)
                cls_explainer.visualize()
            distance = abs(predictions[i][j] - true_labels[i][j])
            total_distance += distance
            index += 1
    total_data_point = len(predictions) * len(predictions[0])
    MAE = total_distance / total_data_point
    print('MAE: ', MAE)
    return predictions

    
def main(model_id, project_name):
    global WITHIN_PROJECT, BATCH_SIZE_RATIO, TOKENIZER, PAD_TOKEN_ID
    # define tokenizer based on model ID
    TOKENIZER = TOKENIZER_MAPPING[model_id]
    PAD_TOKEN_ID = PAD_TOKEN_ID_MAPPING[TOKENIZER]
    
    if len(project_name.split('_')) == 1:
        WITHIN_PROJECT = True
        BATCH_SIZE_RATIO = 0.3
        print('within project inference using model ' + model_id + ' for project ' + project_name)
        file_name = 'sp_dataset/marked_training_data/' + project_name + '.csv'
    else:
        WITHIN_PROJECT = False
        BATCH_SIZE_RATIO = 0.4
        training_project = project_name.split('_')[0]
        testing_project = project_name.split('_')[1]
        print('cross project inference using model ' + model_id + ' trained on ' + training_project 
              + ' for project ' + testing_project)
        file_name = (training_project, testing_project)

    trained_model = load_trained_model(model_id, project_name)
    
    trained_model.to(DEVICE)
    trained_model.eval()
    
    test_dataloader = prepare_test_dataloader(file_name)
    predictions = do_inference(trained_model, test_dataloader)
    
    return predictions

<a id='within_project'></a>
## Within Projects Models

#### There are two parts under Within Project Model section, follow the link to reach the section:
#### 1. [Training Process Inspection](#within_project_tb)
#### 2. [Model Testing](#within_project_model_testing)

#### Different models are available for cross project estimation as follows: 

#### #0 - GPT2 BPE Tokenizer + GPT2
#### #2 - Word-level Story Point Tokenizer + GPT2
#### #6 - WordPiece Story Point Tokenizer + GPT2
#### #7 - SentencePiece Story Point Tokenizer + GPT2 

<a id='within_project_tb'></a>
### Training Process Inspection (TensorBoard)
##### Please specify the model you want to inspect in the cell below within the double quote

In [36]:
# available choices: #0, #1, #2, #3, #4, #5
MODEL_ID = "#0" # < specify your choice in the double quote and keep the quote as it is 
# remember to run the cell after specifying

##### Training process inspection - Run the cell below to inspect the model training process for selected model
##### Note. the cell will open a new tab on your browser

In [37]:
tb_url = "https://tensorboard.dev/experiment/" + TENSORBOARD_MAPPING[MODEL_ID] + "/#scalars"
result = webbrowser.open(tb_url)
if not result:
    print("Please refer to this address to inspect the TensorBoard: \n", tb_url)

Please refer to this address to inspect the TensorBoard: 
 https://tensorboard.dev/experiment/PXtBXB2lTaWHOt7Ba8Lo8A/#scalars


<a id='within_project_model_testing'></a>
### Model Testing
##### Please specify the model and the project you want to use in the cell below within the double quote

In [33]:
# available choices: #0, #1, #2, #3, #4, #5
MODEL_ID = "#0" # < specify your choice in the double quote and keep the quote as it is 

# available choices: appceleratorstudio, aptanastudio, bamboo, clover, datamanagement
#                    duracloud, jirasoftware, mesos, moodle, mule, mulestudio, 
#                    springxd, talenddataquality, talendesb, titanium, usergrid
PROJECT_NAME = "appceleratorstudio" # < specify your choice in the double quote and keep the quote as it is 
# remember to run the cell after specifying

##### Run the cell below to do inference on testing dataset using selected model

In [34]:
# set XAI to False to disable explanation
global XAI 
XAI = True
predictions = main(MODEL_ID, PROJECT_NAME)
print("--- Raw Predictions ---")
print(predictions)

within project inference using model #0 for project appceleratorstudio
Batch Size:  525
using Generic BBPE tokenizer
prediction:  [4.8636594]
true label:  5
Issue Key:  TISTUD-6578
Attribute Values:  [('Update', 0.5185604936489907), ('content', 0.034414294181407096), ('assist', -0.03892219003198505), ('to', 0.0294341892884568), ('support', -0.02352905910255093), ('Fixed', 0.13343850829591242), ('Space', 0.08533973544620435), ('tag', 0.8377869895397267)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
5,5.0 (4.86),N/A,1.58,Update content assist to support Fixed Space tag


prediction:  [4.6614494]
true label:  5
Issue Key:  TISTUD-6579
Attribute Values:  [('Update', 0.40200838859962207), ('content', 0.01836244161100825), ('assist', -0.023801282741257788), ('to', 0.03750430612818244), ('support', -0.0034826219337895773), ('shorthand', 0.008594049626716525), ('notation', -0.015631363392743656), ('for', 0.023957639241280887), ('left', -0.004493334522159187), ('and', 0.027237251215303095), ('right', -0.017267388493209385), ('navigation', 0.012953162911266485), ('buttons', 0.91320682367556)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
5,5.0 (4.66),N/A,1.38,Update content assist to support shorthand notation for left and right navigation buttons


prediction:  [4.8826284]
true label:  5
Issue Key:  TISTUD-6580
Attribute Values:  [('All', 0.4121299900234298), ('oy', 0.03170673121501731), (':', 0.02236975374676784), ('enable', -0.000776166143634095), ('developers', 0.0009035712015075327), ('to', 0.02043246906622741), ('create', 0.02609904750000625), ('new', 0.0002506726917055399), ('widget', 0.0997221781051572), ('components', 0.9042115394342585)]


...

prediction:  [4.6557965]
true label:  5
Issue Key:  TISTUD-8527
Attribute Values:  [('N', 0.361513973681381), ('ort', -0.00010711379364994175), ('on', -0.018355368504812977), ('Security', 0.007897489739931136), ('deems', -0.017972739902664247), ('App', 0.04712107320993544), ('c', 0.014646266721156619), ('Studio', 0.03812212621728549), ('unsafe', 0.0006948106878435705), (',', 0.03148160485017742), ('and', 0.02031960245652077), ('Sym', 0.06079039059476172), ('ant', 0.01468398278202185), ('ec', 0.03875340286672299), ('End', 0.06574321363081011), ('point', 0.026548825039052263), ('Protection', 0.057789963442409595), ('think', 0.00579002180780898), ('the', 0.006400317078947602), ('install', 0.022471091902073796), ('is', 0.027398805000413383), ('unt', 0.011143898088750106), ('rust', -0.006867844508703907), ('worthy', 0.9208754348492743)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
5,5.0 (4.61),N/A,1.74,"N ort on Security deems App c Studio unsafe , and Sym ant ec End point Protection think the install is unt rust worthy"


prediction:  [4.929924]
true label:  5
Issue Key:  TISTUD-8536
Attribute Values:  [('Support', 0.4674020142697461), ('per', 0.024757974008868578), ('-', 0.08375775408754406), ('platform', 0.10597122585781875), ('module', 0.04101654239534982), ('AP', 0.1210498905415875), ('IV', 0.12979374686925585), ('ers', 0.08437718896309893), ('ions', 0.8499269252153899)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
5,5.0 (4.93),N/A,1.91,Support per - platform module AP IV ers ions


MAE:  [0.8377843]


<a id='cross_project'></a>
## Cross Projects Models

#### There are two parts under Cross Project Model section, follow the link to reach the section:
#### 1. [Training Process Inspection](#cross_project_tb)
#### 2. [Model Testing](#cross_project_model_testing)

##### Different models are available for cross project estimation as follows:

### Cross project - within repository models
#### #00 - GPT2 BPE Tokenizer + GPT2
#### #22 - Word-level Story Point Tokenizer + GPT2
#### #66 - WordPiece Story Point Tokenizer + GPT2
#### #77 - SentencePiece Story Point Tokenizer + GPT2  

### Cross project - cross repository models
#### #000 - GPT2 BPE Tokenizer + GPT2
#### #222 - Word-level Story Point Tokenizer + GPT2
#### #666 - WordPiece Story Point Tokenizer + GPT2
#### #777 - SentencePiece Story Point Tokenizer + GPT2 

<a id='cross_project_tb'></a>
### Training Process Inspection (TensorBoard)
##### Please specify the model you want to inspect in the cell below within the double quote

In [38]:
# available choices for within-repository models: #00, #11, #22, #33, #44, #55 
# available choices for cross-repository models: #000, #111, #222, #333, #444, #555
MODEL_ID = "#00" # < specify your choice in the double quote and keep the quote as it is 
# remember to run the cell after specifying

##### Training process inspection - Run the cell below to inspect the model training process for selected model
##### Note. the cell will open a new tab on your browser

In [39]:
tb_url = "https://tensorboard.dev/experiment/" + TENSORBOARD_MAPPING[MODEL_ID] + "/#scalars"
webbrowser.open(tb_url)
if not result:
    print("Please refer to this address to inspect the TensorBoard: \n", tb_url)

Please refer to this address to inspect the TensorBoard: 
 https://tensorboard.dev/experiment/YrgwheHpTeWBIxQjabrtjg/#scalars


<a id='cross_project_model_testing'></a>
### Model Testing
##### Please specify the model and the projects you want to use in the cell below within the double quote

In [47]:
# available choices for within-repository models: #00, #11, #22, #33, #44, #55 
# available choices for cross-repository models: #000, #111, #222, #333, #444, #555
MODEL_ID = "#000" # < specify your choice in the double quote and keep the quote as it is 

# available choices for within-repository projects: mesos_usergrid, usergrid_mesos,
#                                                   appceleratorstudio_aptanastudio, appceleratorstudio_titanium, 
#                                                   titanium_appceleratorstudio, aptanastudio_titanium,
#                                                   mule_mulestudio, mulestudio_mule

# available choices for cross-repository projects: clover_usergrid, talendesb_mesos, 
#                                                  talenddataquality_aptanastudio, mule_titanium,
#                                                  talenddataquality_appceleratorstudio, mulestudio_titanium
#                                                  appceleratorstudio_mulestudio, appceleratorstudio_mule

PROJECT_PAIR = "clover_usergrid" # < specify your choice in the double quote and keep the quote as it is 
# remember to run the cell after specifying

##### Run the cell below to do inference on testing dataset using selected model on selected projects

In [48]:
# set XAI to False to disable explanation
global XAI 
XAI = True
predictions = main(MODEL_ID, PROJECT_PAIR)
print("--- Raw Predictions ---")
print(predictions)

cross project inference using model #000 trained on clover for project usergrid
using GPT2 BPE tokenizer
prediction:  [2.601876]
true label:  3
Issue Key:  USERGRID-16
Attribute Values:  [('Asset', -0.4405357496275028), ('data', -0.2716817268384529), ('does', -0.36614255711827554), ('not', -0.31690760060940787), ('correctly', -0.399397215623702), ('obey', -0.24529450647159942), ('contextual', -0.42461031758092704), ('ownership', -0.2635880258795407), ('like', -0.14574520453904963), ('the', 0.06814945073202608), ('entity', -0.047753334203193824)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
3,3.0 (2.60),N/A,-2.85,Asset data does not correctly obey contextual ownership like the entity


prediction:  [2.7079377]
true label:  3
Issue Key:  USERGRID-17
Attribute Values:  [('Ex', -0.23976819450031572), ('pose', -0.45213950661166535), ('refresh', -0.5270770555571213), ('token', -0.42501991675873907), ('at', -0.3694117620872756), ('the', -0.3310398431874689), ('REST', 0.06224378240016444), ('tier', 0.17234329443630655)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
3,3.0 (2.71),N/A,-2.11,Ex pose refresh token at the REST tier


prediction:  [2.886886]
true label:  3
Issue Key:  USERGRID-19
Attribute Values:  [('Bad', -0.27306454113464707), ('geo', -0.5456011306266084), ('query', -0.6281929952851419), ('returns', -0.4720815162773184), ('entire', -0.0013033404663833274), ('collection', 0.10132159987638246)]


...

prediction:  [2.4128394]
true label:  2
Issue Key:  USERGRID-1244
Attribute Values:  [('Upgrade', -0.3728561542510557), ('push', 0.021271283675281225), ('y', -0.22384171472150244), ('library', -0.30207886181246246), ('to', -0.240688622803028), ('use', -0.34275022446796516), ('latest', -0.2545808426163677), ('version', -0.32201132596533794), ('(', -0.11716885637402814), ('HTTP', -0.29990647971140966), ('/', -0.1812598451766226), ('2', -0.19218821195482141), ('AP', -0.27159230683001506), ('NS', -0.2864350176433692), ('implementation', 0.00944965962272267), (')', 0.023971800552825447), ('', -0.21289062499932065)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,2.0 (2.43),N/A,-3.57,Upgrade push y library to use latest version ( HTTP / 2 AP NS implementation )


prediction:  [2.4382977]
true label:  2
Issue Key:  USERGRID-1275
Attribute Values:  [('Allow', -0.3052381541524623), ('user', -0.1648155774028379), ('grid', -0.2217450487118301), ('.', -0.21402167696459837), ('api', -0.2540071341308374), ('.', -0.20679713852877563), ('url', -0.2585553558609864), ('.', -0.20848237159459557), ('base', -0.21960879819314433), ('to', -0.18806279891754044), ('be', -0.1579279326734375), ('set', -0.22805317787880716), ('when', -0.21174324029431316), ('creating', -0.10224869627336118), ('organizations', -0.1073260280135492), ('.', 0.03813794109528004), ('', -0.5816187745191024)]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,2.0 (2.21),N/A,-3.59,Allow user grid . api . url . base to be set when creating organizations .


MAE:  [0.92141855]
--- Raw Predictions ---
[array([[2.601876 ],
       [2.7079377],
       [2.886886 ],
       [2.6790535],
       [2.2455332],
       [2.281151 ],
       [2.554193 ],
       [2.7148566],
       [2.6917212],
       [2.18256  ],
       [2.6757255],
       [2.3167892],
       [2.3525426],
       [2.2870967],
       [2.6651785],
       [2.6514862],
       [2.578839 ],
       [2.8104508],
       [2.3118763],
       [3.0119553],
       [1.771713 ],
       [2.5814183],
       [2.4524724],
       [2.6156912],
       [2.3299513],
       [2.2546296],
       [2.134285 ],
       [2.695432 ],
       [2.8640854],
       [2.228099 ],
       [2.077245 ],
       [2.647313 ],
       [2.0803676],
       [2.4162085],
       [1.836758 ],
       [2.4836373],
       [2.3479702],
       [2.9654417],
       [2.9306424],
       [2.229202 ],
       [2.8681407],
       [2.902586 ],
       [2.2426398],
       [2.5002944],
       [2.3797948],
       [3.1194544],
       [2.5462675],
       [2.609019